## DASHBOARD INTERACTIVO: ANÁLISIS DE TWEETS SOBRE DESASTRES NATURALES

In [ ]:
import webview
import threading
import dash
from dash import dcc, html, dash_table, Input, Output
import dash_bootstrap_components as dbc
import plotly.express as px
import pandas as pd

# Cargar los datos
file_path = "/content/dataframe_api_historico.csv"
df = pd.read_csv(file_path)

# Procesamiento de datos según la estructura original del modelo
# Filtrar valores no nulos en 'Model2'
df_filtered = df.dropna(subset=["Model2"])
df_filtered = df_filtered[df_filtered["Model2"] != "not informative"]

# Conteo de tipos de desastre
desastres_count = df_filtered["Model2"].value_counts().reset_index()
desastres_count.columns = ["Tipo de Desastre", "Total"]

# Conteo de categorías de información de 'Model3'
info_type_filtered = df.dropna(subset=["Model3"])
info_type_filtered = info_type_filtered[info_type_filtered["Model3"] != "not informative"]
info_type_count = info_type_filtered["Model3"].value_counts().reset_index()
info_type_count.columns = ["Categoría de Información", "Total"]

# Conteo de clasificación de 'Model1'
model1_count = df.dropna(subset=["Model1"])["Model1"].value_counts().reset_index()
model1_count.columns = ["Clasificación", "Total"]

# Inicializar la aplicación Dash con Bootstrap
def create_dash_app():
    app = dash.Dash(__name__, external_stylesheets=[dbc.themes.FLATLY])
    app.layout = dbc.Container([
        dbc.Row([
            dbc.Col(html.H1("Análisis de Tweets sobre Desastres Naturales", style={'color': 'black'}), width=12)
        ], className="mb-4 text-center"),

        dbc.Row([
            dbc.Col(html.H3(f"Total de Tweets analizados: {df.shape[0]}", style={'color': 'black'}), width=12)
        ], className="mb-3 text-center"),

        dbc.Row([
            dbc.Col(dcc.Graph(id="pie_model1", config={"responsive": True}), width=6),
            dbc.Col(dcc.Graph(id="grouped_bar_model2", config={"responsive": True}), width=6),
        ], className="mb-4"),

        dbc.Row([
            dbc.Col(dcc.Graph(id="grouped_bar_model3", config={"responsive": True}), width=6),
        ], className="mb-4"),
    ], fluid=True)

    @app.callback(
        Output("grouped_bar_model2", "figure"),
        Output("grouped_bar_model3", "figure"),
        Output("pie_model1", "figure"),
        Input("grouped_bar_model2", "id")
    )
    def update_graphs(_):
        # Gráfico de barras para tipos de desastre
        grouped_bar_model2 = px.bar(
            desastres_count, x="Tipo de Desastre", y="Total",
            title="Distribución de Tipos de Desastres",
            labels={"Total": "Cantidad"},
            color="Tipo de Desastre",
            text_auto=True
        )

        # Gráfico de barras para categorías de información
        grouped_bar_model3 = px.bar(
            info_type_count, x="Categoría de Información", y="Total",
            title="Distribución de Tipos de Información",
            labels={"Total": "Cantidad"},
            color="Categoría de Información",
            text_auto=True
        )

        # Gráfico de pastel para clasificación de Model1
        pie_model1 = px.pie(
            model1_count, names="Clasificación", values="Total",
            title="Distribución de Informative vs Not Related"
        )

        return grouped_bar_model2, grouped_bar_model3, pie_model1

    return app

# Iniciar la aplicación Dash en un hilo separado
def run_dash():
    app = create_dash_app()
    app.run_server(debug=False, port=8050)

if __name__ == '__main__':
    dash_thread = threading.Thread(target=run_dash)
    dash_thread.daemon = True
    dash_thread.start()

    # Abrir el dashboard en una ventana nativa con pywebview
    webview.create_window("Dashboard TFM", "http://127.0.0.1:8050", width=1200, height=800)
    webview.start()


<IPython.core.display.Javascript object>